<a href="https://colab.research.google.com/github/WonJunPark/BiSeNet_colab_inference/blob/master/BiSeNet_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Tue Sep 29 18:13:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import torch
print(torch.__version__)

1.6.0+cu101


In [ ]:
%cd /content
!git clone https://github.com/CoinCheung/BiSeNet.git

/content
Cloning into 'BiSeNet'...
remote: Enumerating objects: 82, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (52/52), done.
remote: Total 238 (delta 28), reused 69 (delta 21), pack-reused 156
Receiving objects: 100% (238/238), 2.42 MiB | 15.68 MiB/s, done.
Resolving deltas: 100% (96/96), done.


# test

In [ ]:
%cd /content/BiSeNet/
!dir

/content/BiSeNet
configs  datasets  example.png	lib  old  README.md  tensorrt  tools


In [ ]:
!python ./tools/demo.py --model bisenetv2 --weight-path ./model_final_v2.pth --img-path ./example.png

In [ ]:
%cd /content/BiSeNet/weight/
!dir

/content/BiSeNet/weight
model_final_v2.pth


# inference

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
%cd /content/gdrive/My Drive/

/content/gdrive/My Drive


In [ ]:
import os

img_object = 'cars'
img_path = './semantic_test/'+img_object+'/'
img_list = os.listdir(img_path)

In [ ]:
print(img_list)

['18132.png', '18121.png', '18227.png', '18212.png', '18156.png', '18133.png', '18135.png', '18235.png', '18158.png', '18052.png', '18165.png', '18065.png', '18106.png', '18170.png', '18148.png', '18085.png', '18245.png', '18092.png', '18228.png', '18151.png', '18113.png', '18066.png', '18248.png', '18083.png', '18268.png', '18215.png', '18175.png', '18186.png', '18098.png', '18099.png', '18196.png', '18258.png', '18110.png', '18126.png', '18198.png', '18253.png', '18108.png', '18266.png', '18246.png', '18104.png', '18252.png', '18240.png', '18077.png', '18075.png', '18050.png', '18137.png', '18081.png', '18194.png', '18090.png', '18087.png', '18091.png', '18213.png', '18237.png', '18129.png', '18209.png', '18149.png', '18219.png', '18160.png', '18242.png', '18057.png', '18143.png', '18088.png', '18161.png', '18249.png', '18251.png', '18250.png', '18171.png', '18069.png', '18155.png', '18169.png', '18072.png', '18254.png', '18114.png', '18105.png', '18140.png', '18220.png', '18063.png'

In [55]:
%cd /content/BiSeNet
!dir

/content/BiSeNet
configs   example.png  model_final_v2.pth  README.md  result	tools
datasets  lib	       old		   res.jpg    tensorrt


In [56]:
!cd ../gdrive/My\ Drive/semantic_test

## cars

In [59]:
import sys
sys.path.insert(0, '.')
import argparse
import torch
import torch.nn as nn
from PIL import Image
import numpy as np
import cv2
import os
import time

import lib.transform_cv2 as T
from lib.models import model_factory
from configs import cfg_factory

torch.set_grad_enabled(False)
np.random.seed(123)


img_object = 'cars'
img_path = '../gdrive/My Drive/semantic_test/'+img_object+'/'
img_list = os.listdir(img_path)

test_time = time.time()

palette = np.random.randint(0, 256, (256, 3), dtype=np.uint8)

for i in img_list:
    

    # args
    # parse = argparse.ArgumentParser()
    # parse.add_argument('--model', dest='model', type=str, default='bisenetv2',)
    # parse.add_argument('--weight-path', type=str, default='./res/model_final.pth',)
    # parse.add_argument('--img-path', dest='img_path', type=str, default='./example.png',)

    # args = parse.parse_args()
    cfg = cfg_factory['bisenetv2']

    # define model
    net = model_factory[cfg.model_type](19)
    net.load_state_dict(torch.load('./model_final_v2.pth', map_location='cpu'))
    net.eval()
    net.cuda()

    # prepare data
    to_tensor = T.ToTensor(
        mean=(0.3257, 0.3690, 0.3223), # city, rgb
        std=(0.2112, 0.2148, 0.2115),
    )

    im = cv2.imread(img_path+i)[:, :, ::-1]
    im = to_tensor(dict(im=im, lb=None))['im'].unsqueeze(0).cuda()

    # inference
    out = net(im)[0].argmax(dim=1).squeeze().detach().cpu().numpy()

    start = time.time()
    pred = palette[out]
    print(time.time() - start)

    cv2.imwrite('../gdrive/My Drive/semantic_test_result/'+img_object+'/'+i, pred)

print('test time: ',time.time()-test_time)

0.044381141662597656
0.03541302680969238
0.03261613845825195
0.03357195854187012
0.03378891944885254
0.035517215728759766
0.03557872772216797
0.03513813018798828
0.03981280326843262
0.03646397590637207
0.04004812240600586
0.035363197326660156
0.03413200378417969
0.03439188003540039
0.03580904006958008
0.03592109680175781
0.03528118133544922
0.03239893913269043
0.03473615646362305
0.03402590751647949
0.03345990180969238
0.04122424125671387
0.03293418884277344
0.034401655197143555
0.034972429275512695
0.03250908851623535
0.03470349311828613
0.03839230537414551
0.03341388702392578
0.032651424407958984
0.03277707099914551
0.0357663631439209
0.0354464054107666
0.03346085548400879
0.03505206108093262
0.03495645523071289
0.03360295295715332
0.03442955017089844
0.03338789939880371
0.03358578681945801
0.033922433853149414
0.03613901138305664
0.03519630432128906
0.03348898887634277
0.033019065856933594
0.03460049629211426
0.03438520431518555
0.03367209434509277
0.034998416900634766
0.03635454177

In [58]:
import sys
sys.path.insert(0, '.')
import argparse
import torch
import torch.nn as nn
from PIL import Image
import numpy as np
import cv2
import os
import time

import lib.transform_cv2 as T
from lib.models import model_factory
from configs import cfg_factory

torch.set_grad_enabled(False)
np.random.seed(123)


img_object = 'crowded'
img_path = '../gdrive/My Drive/semantic_test/'+img_object+'/'
img_list = os.listdir(img_path)

test_time = time.time()

palette = np.random.randint(0, 256, (256, 3), dtype=np.uint8)

for i in img_list:
    

    # args
    # parse = argparse.ArgumentParser()
    # parse.add_argument('--model', dest='model', type=str, default='bisenetv2',)
    # parse.add_argument('--weight-path', type=str, default='./res/model_final.pth',)
    # parse.add_argument('--img-path', dest='img_path', type=str, default='./example.png',)

    # args = parse.parse_args()
    cfg = cfg_factory['bisenetv2']

    # define model
    net = model_factory[cfg.model_type](19)
    net.load_state_dict(torch.load('./model_final_v2.pth', map_location='cpu'))
    net.eval()
    net.cuda()

    # prepare data
    to_tensor = T.ToTensor(
        mean=(0.3257, 0.3690, 0.3223), # city, rgb
        std=(0.2112, 0.2148, 0.2115),
    )

    im = cv2.imread(img_path+i)[:, :, ::-1]
    im = to_tensor(dict(im=im, lb=None))['im'].unsqueeze(0).cuda()

    # inference
    out = net(im)[0].argmax(dim=1).squeeze().detach().cpu().numpy()

    start = time.time()
    pred = palette[out]
    print(time.time() - start)

    cv2.imwrite('../gdrive/My Drive/semantic_test_result/'+img_object+'/'+i, pred)

print('test time: ',time.time()-test_time)

0.033577680587768555
0.03580021858215332
0.042555809020996094
0.039136409759521484
0.03745293617248535
0.03527498245239258
0.036867380142211914
0.03381490707397461
0.03399515151977539
0.03467512130737305
0.0357513427734375
0.0371706485748291
0.034383296966552734
0.03625059127807617
0.03490257263183594
0.04837536811828613
0.03650403022766113
0.03609514236450195
0.039368391036987305
0.04407858848571777
0.041857242584228516
0.043142080307006836
0.039724111557006836
0.03342747688293457
0.036004066467285156
0.03882765769958496
0.03527355194091797
0.03289461135864258
0.035756826400756836
0.03438067436218262
0.035950422286987305
0.03460955619812012
0.03496885299682617
0.03728365898132324
0.03558754920959473
0.03741765022277832
0.040862083435058594
0.04300212860107422
0.044028282165527344
0.04200148582458496
0.04844260215759277
0.0458986759185791
0.04270052909851074
0.03971099853515625
0.04098033905029297
0.04605507850646973
0.0376279354095459
0.03894805908203125
0.03304004669189453
0.04210495

## pedestrians

In [57]:
import sys
sys.path.insert(0, '.')
import argparse
import torch
import torch.nn as nn
from PIL import Image
import numpy as np
import cv2
import os
import time

import lib.transform_cv2 as T
from lib.models import model_factory
from configs import cfg_factory

torch.set_grad_enabled(False)
np.random.seed(123)


img_object = 'pedestrians'
img_path = '../gdrive/My Drive/semantic_test/'+img_object+'/'
img_list = os.listdir(img_path)

test_time = time.time()

palette = np.random.randint(0, 256, (256, 3), dtype=np.uint8)

for i in img_list:
    

    # args
    # parse = argparse.ArgumentParser()
    # parse.add_argument('--model', dest='model', type=str, default='bisenetv2',)
    # parse.add_argument('--weight-path', type=str, default='./res/model_final.pth',)
    # parse.add_argument('--img-path', dest='img_path', type=str, default='./example.png',)

    # args = parse.parse_args()
    cfg = cfg_factory['bisenetv2']

    

    # define model
    net = model_factory[cfg.model_type](19)
    net.load_state_dict(torch.load('./model_final_v2.pth', map_location='cpu'))
    net.eval()
    net.cuda()

    # prepare data
    to_tensor = T.ToTensor(
        mean=(0.3257, 0.3690, 0.3223), # city, rgb
        std=(0.2112, 0.2148, 0.2115),
    )

    im = cv2.imread(img_path+i)[:, :, ::-1]
    im = to_tensor(dict(im=im, lb=None))['im'].unsqueeze(0).cuda()

    # inference
    out = net(im)[0].argmax(dim=1).squeeze().detach().cpu().numpy()

    start = time.time()
    pred = palette[out]
    print(time.time() - start)

    cv2.imwrite('../gdrive/My Drive/semantic_test_result/'+img_object+'/'+i, pred)

print('test time: ',time.time()-test_time)

0.03402519226074219
0.038458824157714844
0.03824901580810547
0.03586173057556152
0.03797173500061035
0.037572383880615234
0.036596059799194336
0.03354477882385254
0.03385639190673828
0.03450608253479004
0.03730177879333496
0.037023305892944336
0.041231632232666016
0.03473496437072754
0.036165714263916016
0.03614473342895508
0.034394025802612305
0.03366994857788086
0.03490900993347168
0.03420305252075195
0.03613901138305664
0.03425335884094238
0.03301239013671875
0.035158395767211914
0.03447723388671875
test time:  25.634735822677612
